In [1]:
import pandas as pd

In [26]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten

In [3]:
import numpy as np

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [5]:
from gensim.models import Word2Vec, KeyedVectors

In [6]:
from string import punctuation

In [7]:
from nltk.tokenize import word_tokenize

In [8]:
modelo = KeyedVectors.load_word2vec_format("C:\\git\\dados\\nlp\\skip_s100\\skip_s100.txt")

In [9]:
df = pd.read_csv("C:\\git\\dados\\nlp\\imdb-reviews-pt-br.csv", encoding="utf-8")
df.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [10]:
df.dropna( inplace=True )
df[df["text_pt"].isna()]

,id,text_en,text_pt,sentiment


In [11]:
# Create a translation table to remove all punctuation
translation_table = str.maketrans("", "", punctuation)
df["texto_limpo"] = df["text_pt"].str.translate(translation_table).str.lower()
df.head()

,id,text_en,text_pt,sentiment,texto_limpo
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,mais uma vez o sr costner arrumou um filme por...
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,este é um exemplo do motivo pelo qual a maiori...
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,primeiro de tudo eu odeio esses raps imbecis q...
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,nem mesmo os beatles puderam escrever músicas ...
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,filmes de fotos de latão não é uma palavra apr...


In [12]:
def tokenizar( texto ):
    return word_tokenize(texto, language='portuguese')

df["texto_final"] = df["texto_limpo"].apply( tokenizar )
df.head()

,id,text_en,text_pt,sentiment,texto_limpo,texto_final
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,mais uma vez o sr costner arrumou um filme por...,"[mais, uma, vez, o, sr, costner, arrumou, um, ..."
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,este é um exemplo do motivo pelo qual a maiori...,"[este, é, um, exemplo, do, motivo, pelo, qual,..."
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,primeiro de tudo eu odeio esses raps imbecis q...,"[primeiro, de, tudo, eu, odeio, esses, raps, i..."
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,nem mesmo os beatles puderam escrever músicas ...,"[nem, mesmo, os, beatles, puderam, escrever, m..."
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,filmes de fotos de latão não é uma palavra apr...,"[filmes, de, fotos, de, latão, não, é, uma, pa..."


In [13]:
dimensoes = 100
max_palavras = 0

for linha in df["texto_final"]:
    if len(linha) > max_palavras:
        max_palavras = len(linha)
print("Texto com o maior numero de palavras: ", max_palavras)

Texto com o maior numero de palavras:  953


In [14]:
vetor_vazio = np.zeros((dimensoes))

In [15]:
def transformar_tokens_vetores( tokens ):
    lista_vetores = []
    for token in tokens:
        vetor = vetor_vazio
        try:
            vetor = modelo[token]
        except KeyError as err:
            pass
    
        lista_vetores.append( vetor )
    
    for _ in range( max_palavras - len(lista_vetores) ):
        lista_vetores.append( vetor_vazio )
    return np.array(lista_vetores)

transformar_tokens_vetores( df["texto_final"][0] ).shape

(953, 100)

In [16]:
df["X"] = df["texto_final"].apply( transformar_tokens_vetores )

In [17]:
df["X"][0].shape

(953, 100)

In [18]:
df["Y"] = df["sentiment"].replace( {"neg": 0, "pos": 1} ) 

C:\Users\anton\AppData\Local\Temp\ipykernel_19324\574035808.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Y"] = df["sentiment"].replace( {"neg": 0, "pos": 1} )


In [19]:
Y = df["Y"]
X = df["X"]

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 100, test_size=0.2)

In [28]:
modelo = Sequential()
modelo.add( Input( (max_palavras, dimensoes) ) )
modelo.add( Flatten() )
modelo.add( Dense( 128, activation="relu" ) )
modelo.add( Dense( 16, activation="relu" ) )
modelo.add( Dense( 1, activation="sigmoid" ) )
modelo.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 95300)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 128)                 │      12,198,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 16)                  │           2,064 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,200,609 (46.54 MB)

 Trainable params: 12,200,609 (46.54 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
modelo.compile( optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"] )

In [32]:
historico = modelo.fit( X_train, Y_train, epochs = 10, batch_size = 128 )

ValueError: Invalid dtype: object

In [ ]:
modelo.evaluate( X_test, Y_test )

In [ ]:
vetorizador_predict = CountVectorizer( max_features = FEATURES, vocabulary = vocabulario, binary = True )

In [ ]:
texto = "I love music"
bow_predict = vetorizador_predict.fit_transform( [ texto ] )
bow_predict_descompressed = pd.DataFrame.sparse.from_spmatrix( bow_predict , columns = vocabulario ).astype(float)
"positivo" if modelo.predict( bow_predict_descompressed )[0][0] >= 0.5 else "negativo"